In [45]:
import socket
import time
import dns.resolver  # dnspython library for external DNS queries
from collections import OrderedDict


# Root Servers

In [46]:

# Simulated DNS records for root, TLD, and authoritative servers
ROOT_SERVERS = {
    "com": "198.41.0.4",
    "edu": "192.5.5.241",
    "gov": "192.203.230.10",
    "in": "203.119.86.101"
}


# Authoritative Servers

In [47]:

TLD_SERVERS = {
    "com": {
        "example.com": "93.184.216.34",
        "localtest.com": "127.0.0.1",
        "abcd.com": "108.98.98.2",
        "fakecompany.com": "420.420.0.10"
    },
    "edu": {
        "fakeinstitute.edu": "100.5.5.3"
    },
    "gov": {
        "fakegovt.gov": "40.3.3.1"
    },
    "in": {
        "xyz.in": "99.0.0.5"
    }
}


# Update Cache

In [48]:

# Cache with fixed size and TTL management
CACHE_SIZE = 2
DEFAULT_TTL = 5  # Default TTL for each cache entry in seconds
DNS_CACHE = OrderedDict()  # OrderedDict to maintain insertion order for LRU policy

def update_cache(domain_name, ip_address):
    """Add a new entry to the cache with TTL, or update existing entry."""
    current_time = time.time()
    # Check if domain is already in cache and update its expiration
    if domain_name in DNS_CACHE:
        DNS_CACHE.move_to_end(domain_name)
    elif len(DNS_CACHE) >= CACHE_SIZE:
        # Cache is full, remove the oldest entry
        DNS_CACHE.popitem(last=False)
    # Add the entry with a new TTL
    DNS_CACHE[domain_name] = (ip_address, current_time + DEFAULT_TTL)
    


# Acces Cache

In [49]:

def get_from_cache(domain_name):
    """Retrieve an IP from the cache if it exists and has not expired."""
    current_time = time.time()
    # Check if the domain is in cache and validate TTL
    #print(DNS_CACHE)
    if domain_name in DNS_CACHE:
        ip_address, expiration = DNS_CACHE[domain_name]
        if current_time < expiration:
            DNS_CACHE.move_to_end(domain_name)  # Update usage order for LRU policy
            return ip_address
        else:
            # Entry expired, remove it
            del DNS_CACHE[domain_name]
    return None


# Iterative DNS Resolution

In [50]:

def iterative_query(domain_name):
    # Query root server based on TLD
    tld = domain_name.split('.')[-1]
    if tld not in ROOT_SERVERS:
        print(f"Root server does not recognize TLD: {tld}")
        return None
    print(f"Querying root server for TLD '{tld}', IP: {ROOT_SERVERS[tld]}")

    # Query the specific TLD server based on the TLD
    tld_server = TLD_SERVERS.get(tld)
    if tld_server and domain_name in tld_server:
        ip_address = tld_server[domain_name]
        print(f"TLD server resolved domain '{domain_name}' to IP: {ip_address}")
        return ip_address

    # If domain is not found in our TLD servers, return None
    print(f"TLD server could not resolve domain: {domain_name}")
    return None


# DNS Server

In [51]:

def handle_query(data, addr, server):
    domain_name = data.decode('utf-8').strip()
    print(f"Received query for: {domain_name} from {addr}")

    # Step 1: Check in DNS cache and serve if available
    cached_ip = get_from_cache(domain_name)
    if cached_ip:
        print(f"Cache hit for {domain_name}. Sending IP: {cached_ip}")
        server.sendto(cached_ip.encode('utf-8'), addr)
        return

    # Step 2: Perform iterative resolution using simulated root and TLD servers
    ip_address = iterative_query(domain_name)
    if ip_address:
        server.sendto(ip_address.encode('utf-8'), addr)
        # Add to cache with a new TTL
        update_cache(domain_name, ip_address)
    else:
        # Step 3: External DNS query if not found locally or in simulated servers
        try:
            print(f"Domain not found in simulated DNS servers. Querying external DNS for {domain_name}.")
            result = dns.resolver.resolve(domain_name, 'A')
            external_ip = result[0].to_text()
            print(f"External DNS resolved {domain_name} to {external_ip}")
            server.sendto(external_ip.encode('utf-8'), addr)
            # Cache the result with a TTL
            update_cache(domain_name, external_ip)
        except Exception as e:
            print(f"Failed to resolve {domain_name}: {e}")
            server.sendto(b"NOT_FOUND", addr)


In [52]:

def dns_server(timeout_seconds):
    server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:
        server.bind(('0.0.0.0', 5053))  # Cloud instance will use public IP
        server.settimeout(5)  # Set a small timeout for receiving data to make the server non-blocking

        print(f"DNS Server is running on port 5053... (will shut down after {timeout_seconds} seconds)")
        
        start_time = time.time()  # Record the start time
        while True:
            if time.time() - start_time > timeout_seconds:
                print(f"Shutting down server after {timeout_seconds} seconds of inactivity.")
                break

            try:
                data, addr = server.recvfrom(512)
                handle_query(data, addr, server)
            except socket.timeout:
                continue

    except OSError as e:
        print(f"Error: {e}")
    finally:
        server.close()
        print("DNS Server shut down.")


In [57]:

if __name__ == "__main__":
    # Run the server for 60 seconds (or any custom duration)
    dns_server(timeout_seconds=60)


DNS Server is running on port 5053... (will shut down after 60 seconds)
Received query for: nknk from ('127.0.0.1', 56522)
Root server does not recognize TLD: nknk
Domain not found in simulated DNS servers. Querying external DNS for nknk.
Failed to resolve nknk: The DNS query name does not exist: nknk.
Received query for: google.com from ('127.0.0.1', 51556)
Querying root server for TLD 'com', IP: 198.41.0.4
TLD server could not resolve domain: google.com
Domain not found in simulated DNS servers. Querying external DNS for google.com.
External DNS resolved google.com to 142.251.42.14
Received query for: google.com from ('127.0.0.1', 64532)
Cache hit for google.com. Sending IP: 142.251.42.14
Received query for: fakeinstitute.edu from ('127.0.0.1', 55119)
Querying root server for TLD 'edu', IP: 192.5.5.241
TLD server resolved domain 'fakeinstitute.edu' to IP: 100.5.5.3
Received query for: google.com from ('127.0.0.1', 56051)
Querying root server for TLD 'com', IP: 198.41.0.4
TLD server c